# COURSE:   PGP [AI&ML]

## Learner :  Chaitanya Kumar Battula
## Module  : NLP
## Topic   : Chatbot  Module-1

# Tasks

Create a high accuracy NNW model in this module-1 and deploy it in the module-2.

# 1   Import Libraries 

In [1]:
import numpy as np
import random

import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()



from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD


import json
import pickle

In [2]:
words=[]
classes = []
documents = []
ignore_words = ['?', '!']


# 2   Provide a Intents Data File

In [3]:
data_file = open('intents.json').read()
intents = json.loads(data_file)
intents 

{'intents': [{'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day'],
   'responses': ['Hello, Welcome to Simplilearn Properties special discount sale event !!!All our customer care agents are busy!                       Meanwhile, How can I help you?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Nice chatting to you, bye',
    'Till next time'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I understand'],
   'context': ['']},
  {'ta

# 3   Preprocess data 

In [4]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [5]:

# lemmaztize and lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))


# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")

# classes = intents
print (len(classes), "classes", classes)

# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

37 documents
8 classes ['amenties', 'builts area', 'cost', 'flats', 'goodbye', 'greeting', 'location', 'thanks']
73 unique lemmatized words ["'s", ',', '2', 'a', 'amenties', 'an', 'anyone', 'apartment', 'are', 'area', 'available', 'awesome', 'bhk', 'big', 'book', 'built', 'buy', 'bye', 'chatting', 'club', 'cost', 'day', 'do', 'doe', 'facility', 'flat', 'for', 'good', 'goodbye', 'gym', 'have', 'hello', 'helpful', 'helping', 'hey', 'hi', 'hola', 'home', 'house', 'how', 'i', 'in', 'interested', 'is', 'it', 'later', 'like', 'located', 'location', 'me', 'next', 'nice', 'of', 'own', 'pool', 'price', 'property', 'purchase', 'see', 'spacious', 'swimming', 'thank', 'thanks', 'that', 'the', 'there', 'till', 'time', 'to', 'want', 'what', 'where', 'you']


In [6]:
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))

# 4  Create training data

In [7]:
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    
    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

print("Traing data type is:", type(train_x))
print("Target  data type is:", type(train_y))
print("Traing data size is:", len(train_x))
print("Target  data size is:", len(train_y))


Training data created
Traing data type is: <class 'list'>
Target  data type is: <class 'list'>
Traing data size is: 37
Target  data size is: 37


<ipython-input-7-31e8d95c76e9>:25: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  training = np.array(training)


# 5    Build the model

In [8]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))
print(model.summary())




Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               9472      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 520       
Total params: 18,248
Trainable params: 18,248
Non-trainable params: 0
_________________________________________________________________
None


In [9]:
# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=1000, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("Model created")

Epoch 1/1000
8/8 [==============================] - 0s 854us/step - loss: 2.1374 - accuracy: 0.0901
Epoch 2/1000
8/8 [==============================] - 0s 1ms/step - loss: 2.1232 - accuracy: 0.1792
Epoch 3/1000
8/8 [==============================] - 0s 999us/step - loss: 2.0681 - accuracy: 0.1770
Epoch 4/1000
8/8 [==============================] - 0s 999us/step - loss: 2.0298 - accuracy: 0.1295
Epoch 5/1000
8/8 [==============================] - 0s 1000us/step - loss: 1.8934 - accuracy: 0.3266
Epoch 6/1000
8/8 [==============================] - 0s 1ms/step - loss: 1.9039 - accuracy: 0.2438
Epoch 7/1000
8/8 [==============================] - 0s 1000us/step - loss: 1.8372 - accuracy: 0.4272
Epoch 8/1000
8/8 [==============================] - 0s 1ms/step - loss: 1.8284 - accuracy: 0.4208
Epoch 9/1000
8/8 [==============================] - 0s 999us/step - loss: 1.6687 - accuracy: 0.4417
Epoch 10/1000
8/8 [==============================] - 0s 1ms/step - loss: 1.5298 - accuracy: 0.5387
Epoch

8/8 [==============================] - 0s 857us/step - loss: 0.0151 - accuracy: 1.0000
Epoch 163/1000
8/8 [==============================] - 0s 999us/step - loss: 0.0213 - accuracy: 1.0000
Epoch 164/1000
8/8 [==============================] - 0s 859us/step - loss: 0.0291 - accuracy: 1.0000
Epoch 165/1000
8/8 [==============================] - 0s 714us/step - loss: 0.1073 - accuracy: 0.9364
Epoch 166/1000
8/8 [==============================] - 0s 859us/step - loss: 0.0166 - accuracy: 1.0000
Epoch 167/1000
8/8 [==============================] - 0s 714us/step - loss: 0.0059 - accuracy: 1.0000
Epoch 168/1000
8/8 [==============================] - 0s 854us/step - loss: 0.0128 - accuracy: 1.0000
Epoch 169/1000
8/8 [==============================] - 0s 857us/step - loss: 0.0146 - accuracy: 1.0000
Epoch 170/1000
8/8 [==============================] - 0s 716us/step - loss: 0.0060 - accuracy: 1.0000
Epoch 171/1000
8/8 [==============================] - 0s 857us/step - loss: 0.0079 - accuracy: 1.

8/8 [==============================] - 0s 858us/step - loss: 0.0655 - accuracy: 1.0000
Epoch 243/1000
8/8 [==============================] - 0s 713us/step - loss: 0.0097 - accuracy: 1.0000
Epoch 244/1000
8/8 [==============================] - 0s 858us/step - loss: 0.0267 - accuracy: 0.9771
Epoch 245/1000
8/8 [==============================] - 0s 715us/step - loss: 0.0076 - accuracy: 1.0000
Epoch 246/1000
8/8 [==============================] - 0s 1000us/step - loss: 0.0164 - accuracy: 1.0000
Epoch 247/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.0026 - accuracy: 1.0000
Epoch 248/1000
8/8 [==============================] - 0s 2ms/step - loss: 0.0051 - accuracy: 1.0000
Epoch 249/1000
8/8 [==============================] - 0s 857us/step - loss: 0.0151 - accuracy: 1.0000
Epoch 250/1000
8/8 [==============================] - 0s 714us/step - loss: 0.0358 - accuracy: 0.9827
Epoch 251/1000
8/8 [==============================] - 0s 714us/step - loss: 0.0020 - accuracy: 1.000

8/8 [==============================] - 0s 857us/step - loss: 0.0126 - accuracy: 1.0000
Epoch 403/1000
8/8 [==============================] - 0s 855us/step - loss: 0.0038 - accuracy: 1.0000
Epoch 404/1000
8/8 [==============================] - 0s 854us/step - loss: 0.0095 - accuracy: 1.0000
Epoch 405/1000
8/8 [==============================] - 0s 857us/step - loss: 0.0264 - accuracy: 1.0000
Epoch 406/1000
8/8 [==============================] - 0s 713us/step - loss: 0.0048 - accuracy: 1.0000
Epoch 407/1000
8/8 [==============================] - 0s 858us/step - loss: 4.7517e-04 - accuracy: 1.0000
Epoch 408/1000
8/8 [==============================] - 0s 714us/step - loss: 0.0069 - accuracy: 1.0000
Epoch 409/1000
8/8 [==============================] - 0s 857us/step - loss: 0.0011 - accuracy: 1.0000
Epoch 410/1000
8/8 [==============================] - 0s 856us/step - loss: 0.0063 - accuracy: 1.0000
Epoch 411/1000
8/8 [==============================] - 0s 857us/step - loss: 0.0035 - accuracy

8/8 [==============================] - 0s 857us/step - loss: 0.0015 - accuracy: 1.0000
Epoch 562/1000
8/8 [==============================] - 0s 857us/step - loss: 0.0030 - accuracy: 1.0000
Epoch 563/1000
8/8 [==============================] - 0s 857us/step - loss: 0.0078 - accuracy: 1.0000
Epoch 564/1000
8/8 [==============================] - 0s 714us/step - loss: 9.9251e-04 - accuracy: 1.0000
Epoch 565/1000
8/8 [==============================] - 0s 857us/step - loss: 0.0012 - accuracy: 1.0000
Epoch 566/1000
8/8 [==============================] - 0s 857us/step - loss: 5.1573e-04 - accuracy: 1.0000
Epoch 567/1000
8/8 [==============================] - 0s 857us/step - loss: 0.0034 - accuracy: 1.0000
Epoch 568/1000
8/8 [==============================] - 0s 857us/step - loss: 0.0024 - accuracy: 1.0000
Epoch 569/1000
8/8 [==============================] - 0s 714us/step - loss: 3.8968e-04 - accuracy: 1.0000
Epoch 570/1000
8/8 [==============================] - 0s 714us/step - loss: 0.0049 - 

8/8 [==============================] - 0s 1ms/step - loss: 0.0148 - accuracy: 1.0000
Epoch 720/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.0044 - accuracy: 1.0000
Epoch 721/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.0021 - accuracy: 1.0000
Epoch 722/1000
8/8 [==============================] - 0s 714us/step - loss: 6.7080e-04 - accuracy: 1.0000
Epoch 723/1000
8/8 [==============================] - 0s 1ms/step - loss: 4.8293e-04 - accuracy: 1.0000
Epoch 724/1000
8/8 [==============================] - 0s 714us/step - loss: 0.0022 - accuracy: 1.0000
Epoch 725/1000
8/8 [==============================] - 0s 999us/step - loss: 7.3672e-04 - accuracy: 1.0000
Epoch 726/1000
8/8 [==============================] - 0s 714us/step - loss: 0.0029 - accuracy: 1.0000
Epoch 727/1000
8/8 [==============================] - 0s 857us/step - loss: 0.0016 - accuracy: 1.0000
Epoch 728/1000
8/8 [==============================] - 0s 714us/step - loss: 0.0030 - accuracy

8/8 [==============================] - 0s 857us/step - loss: 0.0426 - accuracy: 1.0000
Epoch 878/1000
8/8 [==============================] - 0s 714us/step - loss: 4.6127e-04 - accuracy: 1.0000
Epoch 879/1000
8/8 [==============================] - 0s 714us/step - loss: 0.0151 - accuracy: 1.0000
Epoch 880/1000
8/8 [==============================] - 0s 714us/step - loss: 0.0031 - accuracy: 1.0000
Epoch 881/1000
8/8 [==============================] - 0s 713us/step - loss: 8.7301e-04 - accuracy: 1.0000
Epoch 882/1000
8/8 [==============================] - 0s 855us/step - loss: 9.1511e-05 - accuracy: 1.0000
Epoch 883/1000
8/8 [==============================] - 0s 714us/step - loss: 4.7497e-04 - accuracy: 1.0000
Epoch 884/1000
8/8 [==============================] - 0s 857us/step - loss: 5.7093e-04 - accuracy: 1.0000
Epoch 885/1000
8/8 [==============================] - 0s 714us/step - loss: 5.2359e-04 - accuracy: 1.0000
Epoch 886/1000
8/8 [==============================] - 0s 854us/step - los

8/8 [==============================] - 0s 997us/step - loss: 2.8990e-04 - accuracy: 1.0000
Epoch 957/1000
8/8 [==============================] - 0s 1ms/step - loss: 4.7248e-04 - accuracy: 1.0000
Epoch 958/1000
8/8 [==============================] - 0s 716us/step - loss: 0.0023 - accuracy: 1.0000
Epoch 959/1000
8/8 [==============================] - 0s 858us/step - loss: 3.7245e-04 - accuracy: 1.0000
Epoch 960/1000
8/8 [==============================] - 0s 857us/step - loss: 0.0071 - accuracy: 1.0000
Epoch 961/1000
8/8 [==============================] - 0s 1000us/step - loss: 0.0039 - accuracy: 1.0000
Epoch 962/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.0072 - accuracy: 1.0000
Epoch 963/1000
8/8 [==============================] - 0s 1ms/step - loss: 0.0013 - accuracy: 1.0000
Epoch 964/1000
8/8 [==============================] - 0s 714us/step - loss: 0.0018 - accuracy: 1.0000
Epoch 965/1000
8/8 [==============================] - 0s 999us/step - loss: 7.9124e-04 - a


#### Move to  >> Chatbot_Module-2 _ GUI 

# End Of Chatbot_Module-1_Training